In [23]:
import pandas as pd
import numpy as np

In [20]:
df_1w = pd.read_csv('../database/binance/1w/BTC-USDT-USDT.csv', sep=',')

In [24]:
df_1w['Avg_day'] = ((df_1w['close'] + df_1w['open'] + df_1w['high'] + df_1w['low']) / 4).round(2)
df_1w['Avg_corps'] = ((df_1w['close'] + df_1w['open']) / 2).round(2)
df_1w['Avg_meches'] = ((df_1w['high'] + df_1w['low']) / 2).round(2)
df_1w['Pourc_Price_Evol_14d'] = (100 * df_1w['Avg_day'].shift(-14) / df_1w['Avg_day']).round(2) - 100
df_1w['Label'] = np.where(df_1w['Pourc_Price_Evol_14d'] > 5, 1,np.where(df_1w['Pourc_Price_Evol_14d'] < -5, -1, 0))

In [25]:
df_1w.head(20)

,date,open,high,low,close,volume,Avg_day,Avg_corps,Avg_meches,Pourc_Price_Evol_14d,Label
0,1567382400000,10000.00,10412.65,10000.00,10391.63,3096.291,10201.07,10195.81,10206.33,-28.18,-1
1,1567987200000,10316.62,10475.54,9884.31,10302.22,110846.484,10244.67,10309.42,10179.92,-30.30,-1
2,1568592000000,10302.00,10353.81,9530.02,10023.04,160591.544,10052.22,10162.52,9941.92,-26.24,-1
3,1569196800000,8061.98,10046.91,7700.67,8041.96,279795.272,8462.88,8051.97,8873.79,-14.05,-1
4,1569801600000,8042.08,8499.00,7709.01,7852.79,257976.889,8025.72,7947.44,8104.00,-2.33,0
5,1570406400000,7851.01,8788.00,7764.42,8274.66,367955.166,8169.52,8062.84,8276.21,4.51,0
6,1571011200000,8274.33,8408.62,7812.00,8218.23,508975.286,8178.30,8246.28,8110.31,5.11,1
7,1571616000000,8218.23,10408.48,7172.76,9533.32,881463.175,8833.20,8875.78,8790.62,2.21,0
8,1572220800000,9534.37,9930.13,8933.00,9197.88,697075.617,9398.84,9366.12,9431.56,3.20,0
9,1572825600000,9197.86,9550.00,8669.85,9041.31,665125.812,9114.75,9119.58,9109.92,10.41,1


## NN LTSTM + attention + Convolution (ChatGPT)
---

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Layer, Multiply, Permute, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

# Mécanisme d'attention simple
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)               # (batch, time_steps, 1)
        a = K.softmax(e, axis=1)                            # (batch, time_steps, 1)
        output = x * a                                      # (batch, time_steps, features)
        return K.sum(output, axis=1)                        # (batch, features)

# Paramètres du modèle
time_steps = 100
features = 64

# Entrée
inputs = Input(shape=(time_steps, features))

# Convolution 1D
conv = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(inputs)

# LSTM
lstm_out = LSTM(64, return_sequences=True)(conv)

# Attention
attention_out = AttentionLayer()(lstm_out)

# Couche de sortie
output = Dense(1, activation='sigmoid')(attention_out)

# Modèle
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Afficher le résumé
model.summary()

# Temps : 24.8s

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 100, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 64)        │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 64)             │           164 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,605 (178.14 KB)

 Trainable params: 45,605 (178.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Exemple de données fictives
import numpy as np
X = np.random.rand(1000, time_steps, features)
y = np.random.randint(0, 2, 1000)

model.fit(X, y, epochs=5, batch_size=32)

# Temps : 4.4s

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5113 - loss: 0.6969
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4946 - loss: 0.6932
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5293 - loss: 0.6925
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5358 - loss: 0.6919
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5422 - loss: 0.6916
